In [1]:
from langchain.document_loaders import YoutubeLoader
from langchain.llms import OpenAI
from langchain.chains.summarize import load_summarize_chain

In [2]:
# import os
# os.environ["OPENAI_API_KEY"] = "<your OpenAI API key if not set as env var>"

In [4]:
!pip install -qU pytube youtube-transcript-api 

### 1. Simple Videos

In [5]:
loader = YoutubeLoader.from_youtube_url("https://www.youtube.com/watch?v=QsYGlZkevEg", add_video_info=True)

In [6]:
result = loader.load()

In [7]:
print (type(result))
print (f"Found video from {result[0].metadata['author']} that is {result[0].metadata['length']} seconds long")
print ("")
print (result)

<class 'list'>
Found video from Saturday Night Live that is 224 seconds long

[Document(page_content='LADIES AND GENTLEMEN, PEDRO PASCAL! [ CHEERS AND APPLAUSE ] >> THANK YOU, THANK YOU. THANK YOU VERY MUCH. I\'M SO EXCITED TO BE HERE. THANK YOU. I SPENT THE LAST YEAR SHOOTING A SHOW CALLED "THE LAST OF US" ON HBO. FOR SOME HBO SHOES, YOU GET TO SHOOT IN A FIVE STAR ITALIAN RESORT SURROUNDED BY BEAUTIFUL PEOPLE, BUT I SAID, NO, THAT\'S TOO EASY. I WANT TO SHOOT IN A FREEZING CANADIAN FOREST WHILE BEING CHASED AROUND BY A GUY WHOSE HEAD LOOKS LIKE A GENITAL WART. IT IS AN HONOR BEING A PART OF THESE HUGE FRANCHISEs LIKE "GAME OF THRONES" AND "STAR WARS," BUT I\'M STILL GETTING USED TO PEOPLE RECOGNIZING ME. THE OTHER DAY, A GUY STOPPED ME ON THE STREET AND SAYS, MY SON LOVES "THE MANDALORIAN" AND THE NEXT THING I KNOW, I\'M FACE TIMING WITH A 6-YEAR-OLD WHO HAS NO IDEA WHO I AM BECAUSE MY CHARACTER WEARS A MASK THE ENTIRE SHOW. THE GUY IS LIKE, DO THE MANDO VOICE, BUT IT\'S LIKE A BEDRO

In [9]:
llm = OpenAI(temperature=0)

d:\강의자료\생성형AI로AI애플리케이션개발하기\py311\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [10]:
chain = load_summarize_chain(llm, chain_type="stuff", verbose=False)
chain.run(result)

d:\강의자료\생성형AI로AI애플리케이션개발하기\py311\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'\n\nPedro Pascal thanks the audience for their support and talks about his experience filming "The Last of Us" in a freezing Canadian forest. He reflects on his roles in popular franchises like "Game of Thrones" and "Star Wars" and shares a funny encounter with a fan who wanted him to do the voice of his character from "The Mandalorian." He also talks about his Chilean roots and how his family is proud of his success, but also constantly gives out his phone number. He ends by expressing his gratitude to his parents for bringing him to the US and introduces the musical guest, Coldplay.'

### 2. Long Videos

In [11]:
loader = YoutubeLoader.from_youtube_url("https://www.youtube.com/watch?v=6Ub7Z1AGIuk", add_video_info=True)

In [12]:
result = loader.load()

In [13]:
chain = load_summarize_chain(llm, chain_type="stuff", verbose=False)
chain.run(result)

BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 4097 tokens, however you requested 14286 tokens (14030 in your prompt; 256 for the completion). Please reduce your prompt; or completion length.", 'type': 'invalid_request_error', 'param': None, 'code': None}}

Problem, your transcript/document is too long

In [14]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [15]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=0)

In [16]:
texts = text_splitter.split_documents(result)

In [17]:
chain = load_summarize_chain(llm, chain_type="map_reduce", verbose=True)
chain.run(texts[:4])



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"somehow five years later covet hits and you know what they decide to do they're like you know what let's go all in on creating like safety safety stuff for covert like masks respirators um gloves things like that and they become one of the like core providers right at the start of covet they shifted all their production manufacturing stuff they're like forget the vodka bottles forget the LED screens we need n95 masks and we need respirators and we need you know these testing kits and we need and they started producing this stuff and they started just winning all these contracts [Music] we're live Sean uh we have a bunch of stuff you you have a you have a a full menu over here and so do I yeah all right I want you to look at my menu here and I want you to just pick something off the menu go ahead and I'll Riff Off it we don't prefer p

"\n\nA company shifted their focus to producing safety equipment for the COVID-19 pandemic and became a top provider, winning multiple contracts. During a live discussion, the speaker shares their unique experience with a company that creates artificial snow and reflects on the success of their idea. The company's response to the pandemic is also mentioned. In a separate story, a man in a Mastermind group claims to have multiple successful businesses, including one for mobile COVID testing, but the narrator is unsure if it is too good to be true."

### 3. Multiple Videos

In [18]:
youtube_url_list = ["https://www.youtube.com/watch?v=AXq0QHUwmh8", "https://www.youtube.com/watch?v=EwHrjZxAT7g"]

texts = []

text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=0)

for url in youtube_url_list:
    loader = YoutubeLoader.from_youtube_url(url, add_video_info=True)
    result = loader.load()
    
    texts.extend(text_splitter.split_documents(result))

In [19]:
chain = load_summarize_chain(llm, chain_type="map_reduce", verbose=False)
chain.run(texts)

'\n\nThe video discusses the process and cost of building a backyard golf green, including design considerations, materials, and ongoing maintenance. The speaker shares their personal experience and the cost breakdown of their project. They also mention the ongoing work and future plans for their backyard golf course. In addition, the video features actors discussing their first impressions of each other and their favorite items from Game of Thrones. They also talk about their most prized possession, a tear stick, and their first jobs.'